# eTransafe Omeprazole Heatmaps

(C) 2020 dept of Medical Informatics
Erik M. van Mulligen

In [3]:
from __future__ import print_function
import sys  
sys.path.insert(0, '/Users/mulligen/Workspaces/AllProjects/eTS/src')

import knowledgehub
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, Markdown, clear_output, Javascript
import ipywidgets as widgets
import numpy as np
import seaborn as sns
import pandas
import json
import matplotlib.pyplot as plt
from pprint import pprint

ModuleNotFoundError: No module named 'rdkit'

In [2]:
api = knowledgehub.KnowledgeHubAPI()

## Authentication

In [3]:
username = widgets.Text(
    value='e.vanmulligen@erasmusmc.nl',
    placeholder='Knowledge Hub account',
    description='username:',
    disabled=False
)

password = widgets.Password(
    value='',
    placeholder='Knowledge Hub password',
    description='password:',
    disabled=False
)

loginBtn = widgets.Button(description='Login')
status = widgets.Output()

def on_button_clicked(_):
      # "linking function with output"
      with status:
        # what happens when we press the button
        clear_output()
        if api.login(username.value, password.value) == False:
            print("Failed to login")
        else:
            print("successfully logged in")
            #display(Javascript('Jupyter.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

            
# linking button and function together using a button's method
loginBtn.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([username, password, loginBtn,status])

# Translate compound to SMILES using semantic service

In [4]:
compoundNames = ["omeprazole"]
compoundSmiles = []
for compoundName in compoundNames:
    compound = api.SemanticService().normalize(compoundName, ['RxNorm','smiles'])
    if 'concepts' in compound:
        for concept in compound['concepts']:
            if 'vocabularyId' in concept:
                if concept['vocabularyId'] == 'smiles':
                    compoundSmiles.append(concept['conceptCode'])                   

## Retrieve similar compounds

In [5]:
similar_compounds = api.SimilarityService().get(compoundSmiles[0], nr_results = 20)
similar_compound_concepts = []
if similar_compounds != None:
    if ('search_results' in similar_compounds) and (len(similar_compounds['search_results']) == 1):
        search_result = similar_compounds['search_results'][0]
        #pprint(search_result)
        if 'obj_nam' in search_result:
            for cmp in search_result['obj_nam']:
                compound_concept = api.SemanticService().normalize(cmp, ['RxNorm'])
                similar_compound_concepts.append(compound_concept)
        else:
            print('something wrong in the result object from the similarity service')
else:
    print('similarity service failed')
    
print(f'For {compoundNames[0]} {len(similar_compound_concepts)} similar compounds have been found!')

For omeprazole 1 similar compounds have been found!


## Convert concepts to identifiers

In [6]:
compoundNames = []
 
for similar_compound_concept in similar_compound_concepts:
    for concept in similar_compound_concept['concepts']:
        compoundNames.append(concept['conceptName'])

## Data retrieval

In [14]:
import xlsxwriter

workbook = xlsxwriter.Workbook('/tmp/etox-ae-organ-08102020.xlsx') 
worksheet = workbook.add_worksheet('omeprazole') 
worksheet.write('A1', 'adverse event')
worksheet.write('B1', 'organ')
worksheet.write('C1', 'count')

counts = {}
for study in api.eToxSys().getStudiesByCompoundNames(compoundNames):
    f = study['FINDING']
    if f['finding'] != None and len(f['finding']) > 0 and f['specimenOrgan'] != None:
        key = f['finding'] + '|' + f['specimenOrgan']
        if not key in counts:
            counts[key] = 1
        else:
            counts[key] += 1

row = 0
for key in counts:
    parts = key.split('|')
    row += 1
    worksheet.write(row, 0, parts[0])
    worksheet.write(row, 1, parts[1])
    worksheet.write(row, 2, counts[key])

workbook.close()

retrieving 8983 records
